## Module 1 | Homework Solution

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()['version']['build_hash']


'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

- #### Answer 1: dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

In [8]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
from tqdm.auto import tqdm

/home/marcelonieva/Repos/llm_zoom/llm-zoomcamp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
  es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 164.05it/s]


In [15]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    print(hit['_score'])
    


44.50556
35.433445
33.70974
33.2635
32.589073


- #### Answer 3: 44.50

In [37]:
def elastic_search_f(query, size=3, filter=None):
    es_client = Elasticsearch('http://localhost:9200')
    index_name = "course-questions"
    
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": []
            }
        }
    }

    if filter:
        search_query["query"]["bool"]["filter"].append({
            "term" : filter
        })

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    
    return result_docs

In [38]:
query = "How do copy a file to a Docker container?"
filter = {
    "course": "machine-learning-zoomcamp"
}
results = elastic_search_f(query=query, filter=filter)


In [39]:
import json
print(json.dumps(results, indent=2))

[
  {
    "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
    "section": "5. Deploying Machine Learning Models",
    "question": "How do I debug a docker container?",
    "course": "machine-learning-zoomcamp"
  },
  {
    "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
    "section": "5. Deploying Machine Learning Models",
    "question": "How do I copy files from my local machi

- #### Answer 4: "How do I copy files from a different folder into docker container\u2019s working directory?"

In [41]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [42]:
prompt = build_prompt(query, results)
len(prompt)

1446

- #### Answer 5: 1446

In [43]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [45]:
tokens = encoding.encode(prompt)
len(tokens)

320

- #### Answer 6: 320